In [16]:
# ==================== SETTING UP THE MODEL FROM OPEN AI ====================

import os
import json
import pandas as pd
from dotenv import load_dotenv
from langchain_openai import OpenAI
from openai import OpenAI

# ==================== LODING THE ENV VARIABLES =========================
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

# ==================== INISIATING THE CLIENT =============================
client = OpenAI()

In [17]:
from sqlalchemy import create_engine
import pandas as pd


# Path to your SQLite database file
database_file_path = "./db/test.db"

# Create an engine to connect to the SQLite database
# SQLite only requires the path to the database file
engine = create_engine(f'sqlite:///{database_file_path}')
file_url = "./data/all-states-history.csv"
df = pd.read_csv(file_url).fillna(value = 0)
df.to_sql(
    'all_states_history',
    con=engine,
    if_exists='replace',
    index=False
)

20780

In [23]:
# Creating the two function for the retival of the data from the database
import numpy as np
from sqlalchemy import text

def get_hospitalized_increase_for_state_on_date(state_abbr, specific_date):
    try:
        query = f"""
        SELECT date, hospitalizedIncrease
        FROM all_states_history
        WHERE state = '{state_abbr}' AND date = '{specific_date}';
        """
        query = text(query)

        with engine.connect() as connection:
            result = pd.read_sql_query(query, connection)
        if not result.empty:
            return result.to_dict('records')[0]
        else:
            return np.nan
    except Exception as e:
        print(e)
        return np.nan


def get_positive_cases_for_state_on_date(state_abbr, specific_date):
    try:
        query = f"""
        SELECT date, state, positiveIncrease AS positive_cases
        FROM all_states_history
        WHERE state = '{state_abbr}' AND date = '{specific_date}';
        """
        query = text(query)

        with engine.connect() as connection:
            result = pd.read_sql_query(query, connection)
        if not result.empty:
            return result.to_dict('records')[0]
        else:
            return np.nan
    except Exception as e:
        print(e)
        return np.nan


get_hospitalized_increase_for_state_on_date("AK","2021-03-05")

{'date': '2021-03-05', 'hospitalizedIncrease': 3}

In [24]:
# Execute the function calling against the SQL Database

messages = [
    {"role": "user",
     "content": """ how many hospitalized people we had in Alaska
                    the 2021-03-06?"""
    }
]

tools_sql = [
    {
        "type": "function",
        "function": {
            "name": "get_hospitalized_increase_for_state_on_date",
            "description": """Retrieves the daily increase in
                              hospitalizations for a specific state
                              on a specific date.""",
            "parameters": {
                "type": "object",
                "properties": {
                    "state_abbr": {
                        "type": "string",
                        "description": """The abbreviation of the state
                                          (e.g., 'NY', 'CA')."""
                    },
                    "specific_date": {
                        "type": "string",
                        "description": """The specific date for
                                          the query in 'YYYY-MM-DD'
                                          format."""
                    }
                },
                "required": ["state_abbr", "specific_date"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_positive_cases_for_state_on_date",
            "description": """Retrieves the daily increase in 
                              positive cases for a specific state
                              on a specific date.""",
            "parameters": {
                "type": "object",
                "properties": {
                    "state_abbr": {
                        "type": "string",
                        "description": """The abbreviation of the 
                                          state (e.g., 'NY', 'CA')."""
                    },
                    "specific_date": {
                        "type": "string",
                        "description": """The specific date for the
                                          query in 'YYYY-MM-DD'
                                          format."""
                    }
                },
                "required": ["state_abbr", "specific_date"]
            }
        }
    }
]

In [25]:
response = client.chat.completions.create(
    model="gpt-4o-mini-2024-07-18",
    messages=messages,
    tools=tools_sql,
    tool_choice="auto",
)

response_message = response.choices[0].message
tool_calls = response_message.tool_calls

if tool_calls:
    print (tool_calls)
    
    available_functions = {
        "get_positive_cases_for_state_on_date": get_positive_cases_for_state_on_date,
        "get_hospitalized_increase_for_state_on_date":get_hospitalized_increase_for_state_on_date
    }  
    messages.append(response_message)  
   
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            state_abbr=function_args.get("state_abbr"),
            specific_date=function_args.get("specific_date"),
        )
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": str(function_response),
            }
        ) 
    print(messages)

[ChatCompletionMessageToolCall(id='call_yqOc6AioDUGh60gnQgwOXVT0', function=Function(arguments='{"state_abbr":"AK","specific_date":"2021-03-06"}', name='get_hospitalized_increase_for_state_on_date'), type='function')]
[{'role': 'user', 'content': ' how many hospitalized people we had in Alaska\n                    the 2021-03-06?'}, ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_yqOc6AioDUGh60gnQgwOXVT0', function=Function(arguments='{"state_abbr":"AK","specific_date":"2021-03-06"}', name='get_hospitalized_increase_for_state_on_date'), type='function')]), {'tool_call_id': 'call_yqOc6AioDUGh60gnQgwOXVT0', 'role': 'tool', 'name': 'get_hospitalized_increase_for_state_on_date', 'content': "{'date': '2021-03-06', 'hospitalizedIncrease': 0}"}]


In [26]:
second_response = client.chat.completions.create(
            model="gpt-4o-mini-2024-07-18",
            messages=messages,
        )
print (second_response)

ChatCompletion(id='chatcmpl-9qF9kkZEQRFr4XSBhwfobkOzJdC43', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='On March 6, 2021, there was an increase of 0 hospitalized cases related to COVID-19 in Alaska. However, this does not provide the total number of hospitalized individuals at that time, just the change from the previous day. For exact figures on total hospitalizations, you may need to refer to specific health department reports or databases from that date.', role='assistant', function_call=None, tool_calls=None))], created=1722238344, model='gpt-4o-mini-2024-07-18', object='chat.completion', system_fingerprint='fp_ba606877f9', usage=CompletionUsage(completion_tokens=74, prompt_tokens=95, total_tokens=169))
